# **우울증상 발화문 분류 모델**

###사용모델: Hugging Face BERT

아래 영화리뷰 감정분석 구현 코드를 토대로, 프로젝트에 맞게 일부 수정한 코드입니다. 
코드 출처: https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=kLtsS3wB9_52

In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 134.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
import torch, gc
torch.cuda.is_available()
gc.collect()
torch.cuda.empty_cache()


## **Data Load**

In [ ]:
# data import 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 증상 데이터 
train = pd.read_csv('/content/drive/MyDrive/2023_연구학점/jihee/BackTraslation/smyp9/smpy9 - train.csv',encoding='utf-8')

test = pd.read_csv('/content/drive/MyDrive/2023_연구학점/jihee/BackTraslation/smyp9/smpy9 - test.csv', encoding='utf-8')

In [ ]:
print("학습 데이터셋: ",train.shape) 
print("검증 데이터셋: ",test.shape) 

학습 데이터셋:  (398, 2)
검증 데이터셋:  (99, 2)


In [ ]:
# 라벨별 순서가 뒤섞이도록 shuffle
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

In [ ]:
train

,Text,Label
0,우울증인지는 모르겠지만 어렸을 때부터 자살에 대해 생각해왔고 자살을 시도하기 직전에...,1
1,안녕하세요 전 20살 여자아이입니다현재 우울증이 심각하고 공황장애와 불면증으로 고...,1
2,거식증 폭식증 이러한 식이장애들이 낳은 우울증이 고등학생 때부터 줄 곧 있었어요1년...,1
3,작년 12월 훈련병 시기에 평소 수면장애로 인해 규칙적인 생활이 어려운 편이었는데 ...,1
4,"남자친구 당신은 정말 좋은 사람이고, 내가 이런 사람이라서 미안해요, 당신이 아직 ...",1
...,...,...
393,맞아요. 주문할 때 목소리좀 크게 해주세요. 그리고 문 열고 들어 오면서 말하지 말...,0
394,남자친구야너는 정말 좋은 사람인데내가 이런 사람이라서 미안해자꾸 죽고싶다고만 생각하...,1
395,"이런 섭식장애로 인한 거식증, 폭식증, 우울증이 고등학교 때부터 줄을 잇고 있습니다...",1
396,20대 초반의 여대생입니다.자살충동이 자꾸 일어나 너무 괴롭습니다.지금은 휴학중이구...,1


In [ ]:
test

,Text,Label
0,가게에서 피자 해 먹네요!,0
1,상해 안 가는 사람은 삶의 의욕을 잃네요.,0
2,안녕하세요올해 고3이 되는 19살 학생입니다.이렇게 용기내어 지식in에 글을 써보려...,1
3,현재 아버지께서 극심한 우울증으로 자살을 이야기 하고 계십니다.정신과 상담을 받...,1
4,음. 비트코인만큼이나 리니지의 아덴도 역사 깊은 대체 화폐려나요?,0
...,...,...
94,언니 오늘 생일이었네요! 생일 축하해요!,0
95,죽고싶다는 생각 그니까 충동적인 생각이 아닌 몇년동안 나에게 그 무엇을 주든 행복이...,1
96,안녕하세요현재 해군 부사관으로 복무중입니다.입대전 우울증 진단받고 치료를하다가 괜찮...,1
97,이건 완전 미친사람이 따로 없네요...멀쩡하다가 갑자기 가슴이 옥죄어 오면서눈물이 ...,1


## **Data Processing - Train Data**

In [ ]:
sentences = train['Text']
sentences[:10]

0    우울증인지는 모르겠지만 어렸을 때부터 자살에 대해 생각해왔고 자살을 시도하기 직전에...
1     안녕하세요 전 20살 여자아이입니다현재 우울증이 심각하고 공황장애와 불면증으로 고...
2    거식증 폭식증 이러한 식이장애들이 낳은 우울증이 고등학생 때부터 줄 곧 있었어요1년...
3    작년 12월 훈련병 시기에 평소 수면장애로 인해 규칙적인 생활이 어려운 편이었는데 ...
4    남자친구 당신은 정말 좋은 사람이고, 내가 이런 사람이라서 미안해요, 당신이 아직 ...
5             이타치 취미가 맛집 탐방이에요? 아, 귀여워요. 맛있는 거 많이 먹어요.
6                                             라멘 맛있어요.
7                    떡볶이 너무 먹고싶은데 저랑 떡볶이 먹으러 가실 분 구해요.
8    병원... 가보려고 그랬는데 처음 오신분은 안받는대서 허무하게 나왔다 진짜 너무 힘...
9                                     으아, 고양이 키우고 싶어요!
Name: Text, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 우울증인지는 모르겠지만 어렸을 때부터 자살에 대해 생각해왔고 자살을 시도하기 직전에 저는 트랜스젠더이고 어렸을 때부터 알고 있었습니다 치료를 시작한 이후로 현실 때문에 더 나쁜 생각을 해왔고 정말 살고 싶지 않습니다. 저 같은 경우에는 정신과 치료를 받고 자살 생각이 없고 긍정적으로 변한다면 이런 현실을 계속 살 것 같아요? 현실에 부딪혀 다칠까봐 더 무섭고 정신과 의사를 찾아가서 내 인생을 뛰어넘는 치료를 받는 게 두렵고 이 감정을 유지한다면, 무엇을 하든, 용기와 당혹스러운 탈출 방법을 갖게 될 것입니다... 하지만 여러분은 보통 이런 우울한 감정이나 자살 생각을 가지고 정신과 의사들을 방문합니까? 당신은 당신의 지인이나 가족에 의해 강제로 가야 합니까? 모르겠어요, 하지만 궁금해요... [SEP]',
 '[CLS] \xa0안녕하세요 전 20살 여자아이입니다현재 우울증이 심각하고 공황장애와 불면증으로 고생하고 있습니다하지만 아는 지인의 가계에서 일을 하고밤에 소주로 술 기운을 빌려 자살시도도 많이 해봤습니다의사를 찾아갔고 약도 처방을 받아 먹고 있습니다만소용이 없습니다의사한테 의문이 갈 정도입니다 약은 매번 다 잘 챙겨 먹었고..했는대오히려 더 안 좋아 졌습니다의사는 입원을 권유 했지만 거절하고 내일 병원을 갑니다 더 심하면 자세한 검사가 필요하다라고 말 이 나온 상태입니다 매일 밤마다 꿈을 꾸고 일어나면 개운한 적이 없습니다 몇 년 전 인거 같아여그림 검사 질문 문항 검사 다면적검사 까지 했습니다...정말 죽어야 되는대 살아서 벌을 받는 건가 생각이 될 정도입니다 도와주세여 ..전 어떻게 하면되져.?\xa0 [SEP]',
 '[CLS] 거식증 폭식증 이러한 식이장애들이 낳은 우울증이 고등학생 때부터 줄 곧 있었어요1년을 집밖으로 안나간다던지다시 잘 생활하다가도 또 세상과의 단절을 반복해요2년전 아빠가 돌아가신 뒤로 점점 더 심해지는거 같아요작은 타격에도 제 모든 일상이 무너져요직장을 그만둬야 할 정도로요새롭게 생긴 증상이 있다면아침에 눈을 

In [ ]:
# 라벨 추출
labels = train['Label'].values
len(labels)

398

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 우울증인지는 모르겠지만 어렸을 때부터 자살에 대해 생각해왔고 자살을 시도하기 직전에 저는 트랜스젠더이고 어렸을 때부터 알고 있었습니다 치료를 시작한 이후로 현실 때문에 더 나쁜 생각을 해왔고 정말 살고 싶지 않습니다. 저 같은 경우에는 정신과 치료를 받고 자살 생각이 없고 긍정적으로 변한다면 이런 현실을 계속 살 것 같아요? 현실에 부딪혀 다칠까봐 더 무섭고 정신과 의사를 찾아가서 내 인생을 뛰어넘는 치료를 받는 게 두렵고 이 감정을 유지한다면, 무엇을 하든, 용기와 당혹스러운 탈출 방법을 갖게 될 것입니다... 하지만 여러분은 보통 이런 우울한 감정이나 자살 생각을 가지고 정신과 의사들을 방문합니까? 당신은 당신의 지인이나 가족에 의해 강제로 가야 합니까? 모르겠어요, 하지만 궁금해요... [SEP]
['[CLS]', '우', '##울', '##증', '##인', '##지는', '모', '##르', '##겠', '##지만', '어', '##렸', '##을', '때', '##부터', '자', '##살', '##에', '대해', '생', '##각', '##해', '##왔', '##고', '자', '##살', '##을', '시', '##도', '##하기', '직', '##전에', '저', '##는', '트', '##랜', '##스', '##젠', '##더', '##이고', '어', '##렸', '##을', '때', '##부터', '알', '##고', '있', '##었', '##습', '##니다', '치', '##료', '##를', '시', '##작', '##한', '이후', '##로', '현', '##실', '때문에', '더', '나', '##쁜', '생', '##각', '##을', '해', '##왔', '##고', '정', '##말', '살', '##고', '싶', '##지', '않', '##습', '##니다', '.', '저', '같은', '경우', '##에는', '정', '##신', '##과', '치', '##료', '##를', '받고', '자',

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

###///////공부!!!
# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

###///////공부!!!
# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9604,  78123, 119230,  12030,  32815,   9283,  31401,
       118632,  28578,   9546, 118881,  10622,   9137,  17655,   9651,
       106249,  10530,  33378,   9420,  66540,  14523, 119163,  11664,
         9651, 106249,  10622,   9485,  12092,  22440,   9707,  68767,
         9663,  11018,   9890, 118856,  12605, 119208,  54141,  54355,
         9546, 118881,  10622,   9137,  17655,   9524,  11664,   9647,
       119138, 119081,  48345,   9779,  38688,  11513,   9485,  38709,
        11102,  18347,  11261,   9978,  31503,  20729,   9074,   8982,
       119023,   9420,  66540,  10622,   9960, 119163,  11664,   9670,
        89523,   9408,  11664,   9495,  12508,   9523, 119081,  48345,
          119,   9663,  18589,  28467,  15303,   9670,  25387,  11882,
         9779,  38688,  11513,  79602,   9651, 106249,   9420,  66540,
        10739,   9555,  11664,   8931,  16605,  17022,   9352,  14102,
        14867,  80956,   9978,  83380,  77039,   9408,   8870,   8855,
      

In [ ]:
#어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9448,  52951,   9691,  29455,   9952,  26737,  11664,   9640,
         21876,  82881,   9737,  41850,   9641,  21789,   9448,  52951,   9689,
         17730,  10530,   8843,  45465,   9489,  18227,  12030,  11287,  48549,
           119,   9625,  12692,  28000,  96279,   9330,  78123, 118671,  48549,
           136,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
# 배치 사이즈
batch_size = 50

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

#Data Processing - Test Data

In [ ]:
#  문장 추출
sentences = test['Text']
sentences[:10]

0                                       가게에서 피자 해 먹네요!
1                              상해 안 가는 사람은 삶의 의욕을 잃네요.
2    안녕하세요올해 고3이 되는 19살 학생입니다.이렇게 용기내어 지식in에 글을 써보려...
3      현재 아버지께서 극심한 우울증으로 자살을 이야기 하고 계십니다.정신과 상담을 받...
4                 음. 비트코인만큼이나 리니지의 아덴도 역사 깊은 대체 화폐려나요?
5    안녕하세요 초등여학생입니다. 제가 우울증인가? 싶어서 우울증 테스트와 ㅈ ㅏ살 테스...
6                                        밖에 비오네요. 주륵주륵
7                                    저 집 와이파이가 자꾸 끊겨요.
8    애기 낳기전부터 우울감은 어릴적부터 있었던거같아요 애기 낳고 조리원 퇴소하는날 차안...
9    꼬맹이 들려 보낼 김밥이랑 초밥, 도시락 싸느라 손이 팅팅 불었어요. 일식하는 분들...
Name: Text, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 가게에서 피자 해 먹네요! [SEP]',
 '[CLS] 상해 안 가는 사람은 삶의 의욕을 잃네요. [SEP]',
 "[CLS] 안녕하세요올해 고3이 되는 19살 학생입니다.이렇게 용기내어 지식in에 글을 써보려고 합니다.요즘 고민이 너무 많아지는데요 그 고민 중 가장 심각하게 생각하는 것은바로 우울증입니다.무엇을 해도 우울하고 의욕이 없어요..... 19살이라는 나이에 맞지 않게우울증이 심한것 같아요 확실히 말하면 우울증이라는 진단을 병원에서 받지는 않았지만저 자신이 제일 잘 알잖아요?하루종일 씻지도 않고 그것도 3일동안.... 밖에 나가지도 않아요오로지 잠 핸드폰 잠 핸드폰 이런 생활이 벌써 한달이 되어가고 있네요공부는 통 손에 잡히지도 않습니다....한번은 이런 생각이 들더군요'19살인데도 이렇게 젊은 나이인데도 살기가 싫고 사는게 재미없다면차라리 죽는게 나을것 같다고'머리속으로 자살생각을 수없이 많이 했습니다...심지어 계획을 짜기도 했고요하지만 남겨질 아빠와 엄마를 생각하면 가슴이 아픕니다...열심히 살고 싶어요 최선을 다해서 태어난 의무를 다하고 싶습니다저에게 조언을 남겨주세요! [SEP]",
 '[CLS] \xa0 현재 아버지께서 극심한 우울증으로 자살을 이야기 하고 계십니다.정신과 상담을 받기도 했지만 아주 심한 상태이신듯 합니다잠을 거의 주무시지 못하며평소 85키로 정도 나갔는데 최근 급격히 10kg정도 빠지셨습니다또한 최근 전립선암(초기) 판정을 받으셨습니다.우울증이 가장 심각한 문제 인듯 합니다가족으로써 할 수 있는 방법을 가르쳐 주시면 감사하겠습니다 [SEP]',
 '[CLS] 음. 비트코인만큼이나 리니지의 아덴도 역사 깊은 대체 화폐려나요? [SEP]',
 '[CLS] 안녕하세요 초등여학생입니다. 제가 우울증인가? 싶어서 우울증 테스트와 ㅈ ㅏ살 테스트를했는데 다 병원 방문해보세요,상담해보세요,상당한 우울증입니다등등 다 심한걸로 나오는데 가족,친구에게 말하기엔 그사람들 때문에 죽고싶어서 말하지 못해 여기 써보아요 [SEP]'

In [ ]:
# 라벨 추출
labels = test['Label'].values
len(labels)

99

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 가게에서 피자 해 먹네요! [SEP]
['[CLS]', '가', '##게', '##에서', '피', '##자', '해', '먹', '##네', '##요', '!', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101,  8843, 14153, 11489,  9946, 13764,  9960,  9266, 77884,
       48549,   106,   102,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101,  8843, 14153, 11489,  9946, 13764,  9960,  9266, 77884, 48549,
          106,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
# 배치 사이즈
batch_size = 20

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

#모델생성

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
#디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 2

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#모델학습

In [ ]:
from sklearn import metrics

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
 
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    

    print("f1 score: " , metrics.f1_score(labels_flat, pred_flat))

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 출력 로짓 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...

  Average training loss: 0.55
  Training epcoh took: 0:00:09

Running Validation...
f1 score:  0.9
  Accuracy: 0.90
  Validation took: 0:00:00

======== Epoch 2 / 2 ========
Training...

  Average training loss: 0.33
  Training epcoh took: 0:00:07

Running Validation...
f1 score:  0.9
  Accuracy: 0.90
  Validation took: 0:00:00

Training complete!
